In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from analysis_utils import *

In [2]:
data = pd.read_csv('../results/Main/summary.csv')
data_robustness = pd.read_csv('../results/Rebuttal/summary.csv')

data = pd.concat([data,data_robustness],axis=0)

# data = data.iloc[-21:,:]
total_elapsed_time = data['time_elapsed'].sum()

data = data[['method','test_loss_mean','test_loss_std','dataset_name']]


data.loc[:,'core_dataset_name'] = data['dataset_name'].apply(extract_core_dataset_name)
synthetic_datasets = ['SIR','PK','synthetic-tumor','HIV']
# data = data[data['core_dataset_name'].isin(synthetic_datasets)]

data.loc[:,'noise_ratio'] = data['dataset_name'].apply(extract_noise_ratio)


data = data[['method','core_dataset_name','noise_ratio','test_loss_mean','test_loss_std']]
data.columns = ['Method','Dataset','Noise Ratio','Test Loss Mean','Test Loss Std']

# Filter out real datasets. Only use rows where Dataset is in the following list


# Create a latex table where each row is a method and each column is a dataset
# For each dataset, have a separate subcolumn for each noise ratio
# Each value should show the mean test loss and the standard deviation a a subscript in parentheses

datasets = data['Dataset'].unique()
methods = data['Method'].unique()
noise_ratios = data['Noise Ratio'].unique()

# Construct two-level column names
column_names = []
for dataset in datasets:
    for noise_ratio in noise_ratios:
        column_names.append((dataset,noise_ratio))

latex_table = pd.DataFrame(index=methods,columns=pd.MultiIndex.from_tuples(column_names))

for dataset in datasets:
    for method in methods:
        for noise_ratio in noise_ratios:
            mask = (data['Dataset']==dataset) & (data['Method']==method) & (data['Noise Ratio']==noise_ratio)
            if mask.sum() > 0:
                mean = data.loc[mask,'Test Loss Mean'].values[0]
                std = data.loc[mask,'Test Loss Std'].values[0]
                formatted_noise = f"{std:.3f}".lstrip('0')
                latex_table.loc[method,(dataset,noise_ratio)] = fr"${mean:.3f}_"+"{("+formatted_noise+r")}$"



dataset_dict = {
    'simple':'Logistic Growth',
    'general':'General ODE',
    'tacrolimus':'Pharmacokinetic',
    'mackey':'Mackey-Glass',
    'integral':'Integro-DE',
    'duffing':'Duffing',
    'tumor-real':'Tumor growth (real)',
    'tacrolimus-real':'Drug concentration (real)',
}

method_dict = {
    'NeuralODE':'Neural ODE',
    'NeuralLaplace':'Neural Laplace',
    'DeepONet':'DeepONet',
    'SemanticODE-default':'Semantic ODE',
    'WeakSINDy-5':'WSINDy-5',
    'SINDy-0':'SINDy',
    'WeakSINDy-0':'WSINDy'
    }

In [3]:
latex_table

SIR                     tacrolimus-real       \
                               0.01              0.10              0.01 0.10   
NeuralODE          $0.021_{(.006)}$  $0.102_{(.001)}$  $0.267_{(.034)}$  NaN   
ANODE              $0.017_{(.003)}$  $0.101_{(.001)}$  $0.250_{(.011)}$  NaN   
LatentODE          $0.016_{(.002)}$  $0.101_{(.001)}$  $0.260_{(.013)}$  NaN   
SINDy-5            $0.014_{(.001)}$  $0.135_{(.007)}$  $0.316_{(.032)}$  NaN   
SINDy-20           $0.015_{(.001)}$  $0.142_{(.003)}$  $0.346_{(.040)}$  NaN   
WSINDy-5           $0.026_{(.025)}$  $0.120_{(.006)}$  $0.415_{(.190)}$  NaN   
WSINDy-20          $0.016_{(.001)}$  $0.144_{(.010)}$  $0.816_{(.670)}$  NaN   
PSODE-more         $0.026_{(.004)}$  $0.103_{(.001)}$  $0.256_{(.007)}$  NaN   
PSODE-biases_more  $0.012_{(.000)}$  $0.100_{(.001)}$  $0.256_{(.007)}$  NaN   

                                              PK  \
                                            0.01   
NeuralODE                       $0.156_{(.006)}$   
ANODE                           $0.147_{(.005)}$   
LatentODE                       $0.145_{(.004)}$   
SINDy-5                         $0.211_{(.013)}$   
SINDy-20                        $0.180_{(.007)}$   
WSINDy-5                        $0.279_{(.009)}$   
WSINDy-20          $9022279.902_{(7885898.610)}$   
PSODE-more                      $0.088_{(.008)}$   
PSODE-biases_more               $0.095_{(.039)}$   

                                                   synthetic-tumor  \
                                            0.10              0.01   
NeuralODE                       $0.182_{(.006)}$  $0.300_{(.063)}$   
ANODE                           $0.184_{(.009)}$  $0.317_{(.049)}$   
LatentODE                       $0.172_{(.003)}$  $0.310_{(.051)}$   
SINDy-5                         $0.249_{(.010)}$  $0.095_{(.018)}$   
SINDy-20               $28593.670_{(57186.893)}$  $0.062_{(.015)}$   
WSINDy-5                        $0.309_{(.016)}$  $0.094_{(.024)}$   
WSINDy-20          $6538986.108_{(7746561.551)}$  $0.056_{(.014)}$   
PSODE-more                      $0.132_{(.013)}$  $0.144_{(.032)}$   
PSODE-biases_more               $0.146_{(.037)}$  $0.098_{(.014)}$   

                                                    bike-sharing       \
                               0.10                         0.01 0.10   
NeuralODE          $0.334_{(.048)}$             $0.178_{(.013)}$  NaN   
ANODE              $0.335_{(.049)}$             $0.154_{(.011)}$  NaN   
LatentODE          $0.338_{(.052)}$             $0.149_{(.006)}$  NaN   
SINDy-5            $0.151_{(.012)}$             $0.267_{(.010)}$  NaN   
SINDy-20           $0.138_{(.013)}$             $0.250_{(.012)}$  NaN   
WSINDy-5           $0.145_{(.017)}$  $412372.371_{(824744.044)}$  NaN   
WSINDy-20          $0.129_{(.008)}$             $0.310_{(.069)}$  NaN   
PSODE-more         $0.290_{(.147)}$             $0.192_{(.013)}$  NaN   
PSODE-biases_more  $0.165_{(.016)}$             $0.195_{(.029)}$  NaN   

                             Beta_2                                   HIV  \
                               0.01 0.10                             0.01   
NeuralODE          $0.028_{(.008)}$  NaN                 $0.188_{(.009)}$   
ANODE              $0.019_{(.003)}$  NaN                 $0.188_{(.008)}$   
LatentODE          $0.022_{(.007)}$  NaN                 $0.183_{(.006)}$   
SINDy-5            $0.390_{(.015)}$  NaN                 $0.175_{(.070)}$   
SINDy-20           $0.361_{(.018)}$  NaN    $3186154.693_{(1929563.129)}$   
WSINDy-5           $1.471_{(.209)}$  NaN                 $0.246_{(.082)}$   
WSINDy-20          $1.742_{(.535)}$  NaN  $29905817.974_{(59631265.783)}$   
PSODE-more         $0.249_{(.227)}$  NaN                 $0.088_{(.006)}$   
PSODE-biases_more  $0.249_{(.227)}$  NaN                 $0.089_{(.006)}$   

                                                    
                                              0.10  
NeuralODE                         $0.214_{(.006)}

In [4]:
datasets_in_order = ['SIR','HIV','PK','synthetic-tumor','tacrolimus-real','bike-sharing']
latex_table = latex_table.loc[:,datasets_in_order]

In [ ]:
# Save as a latex table
with open('output/tab_main_results/main_results.tex', 'w') as f:
    f.write(latex_table.to_latex(escape=False))

In [ ]:
markdown_table = dataframe_to_markdown(flatten_noise_level(latex_table))
with open('output/tab_main_results/main_results.md', 'w') as f:
    f.write(markdown_table)